# License Plate Detection and Redaction
## Part of Aegis - Image Anonymization System

This notebook demonstrates how to detect and redact license plates in images to protect vehicle privacy and prevent identification.

## Setup and Imports

In [ ]:
# Add parent directory to path to import Aegis modules
import sys
sys.path.append('..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from Aegis.plate_redactor import LicensePlateRedactor

## Helper Functions for Visualization

In [ ]:
def display_comparison(original, processed, num_plates):
    """
    Display original and processed images side by side.
    
    Args:
        original: Original image (BGR)
        processed: Processed image (BGR)
        num_plates: Number of plates detected
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    
    # Convert BGR to RGB for matplotlib
    original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    processed_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
    
    axes[0].imshow(original_rgb)
    axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(processed_rgb)
    axes[1].set_title(f'Redacted Image\n({num_plates} plate(s) detected)', 
                     fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

def display_single_image(image, title='Image'):
    """
    Display a single image.
    
    Args:
        image: Image to display (BGR)
        title: Title for the image
    """
    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title, fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## 1. Basic Usage - Blur Method

The blur method applies Gaussian blur to detected license plates.

In [ ]:
# Initialize redactor with blur method
redactor = LicensePlateRedactor(method='blur')

# Load test image
input_path = '../assets/targets/sample_car.jpg'  # Update with your image path
image = cv2.imread(input_path)

if image is None:
    print(f"Error: Could not load image from {input_path}")
else:
    # Process image
    processed, num_plates = redactor.redact(image, strength=51)
    
    print(f"✓ Detected and redacted {num_plates} license plate(s)")
    
    # Display results
    display_comparison(image, processed, num_plates)

## 2. Pixelation Method

The pixelate method creates a mosaic effect on license plates.

In [ ]:
# Initialize redactor with pixelation method
redactor_pixelate = LicensePlateRedactor(method='pixelate')

if image is not None:
    # Process with pixelation (strength controls pixel size)
    processed_pixelated, num_plates = redactor_pixelate.redact(image, strength=10)
    
    print(f"✓ Pixelated {num_plates} license plate(s)")
    
    # Display results
    display_comparison(image, processed_pixelated, num_plates)

## 3. Blackout Method

The blackout method completely blacks out detected license plates.

In [ ]:
# Initialize redactor with blackout method
redactor_black = LicensePlateRedactor(method='black')

if image is not None:
    # Process with blackout
    processed_black, num_plates = redactor_black.redact(image)
    
    print(f"✓ Blacked out {num_plates} license plate(s)")
    
    # Display results
    display_comparison(image, processed_black, num_plates)

## 4. Visualize Detection Boxes

Show detection boxes before redaction for debugging.

In [ ]:
if image is not None:
    # Show detection boxes
    image_with_boxes, num_plates = redactor.redact(image, show_detections=True)
    
    print(f"✓ Detected {num_plates} license plate(s)")
    
    # Display
    display_single_image(image_with_boxes, 'Detected License Plates (Green Boxes)')

## 5. Adjust Redaction Strength

Compare different blur strengths.

In [ ]:
if image is not None:
    # Test different blur strengths
    strengths = [25, 51, 101]
    
    fig, axes = plt.subplots(1, len(strengths), figsize=(18, 6))
    
    for idx, strength in enumerate(strengths):
        processed, _ = redactor.redact(image.copy(), strength=strength)
        processed_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
        
        axes[idx].imshow(processed_rgb)
        axes[idx].set_title(f'Blur Strength: {strength}', fontsize=12, fontweight='bold')
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

## 6. Process and Save Single Image

In [ ]:
# Process and save to file
input_file = '../assets/targets/sample_car.jpg'
output_file = '../assets/targets/redacted_car.jpg'

try:
    num_plates = redactor.process_image(input_file, output_file, strength=51)
    print(f"✓ Successfully processed image")
    print(f"✓ Detected and redacted {num_plates} license plate(s)")
    print(f"✓ Saved to: {output_file}")
except Exception as e:
    print(f"✗ Error: {e}")

## 7. Batch Processing

Process multiple images in a directory.

In [ ]:
# Process all images in a directory
input_directory = '../assets/targets/input_images'
output_directory = '../assets/targets/output_images'

try:
    processed_count, total_plates = redactor.process_directory(
        input_directory,
        output_directory,
        strength=51
    )
    
    print(f"\n{'='*50}")
    print(f"Batch Processing Complete")
    print(f"{'='*50}")
    print(f"✓ Processed: {processed_count} images")
    print(f"✓ Total plates detected: {total_plates}")
    print(f"✓ Output saved to: {output_directory}")
    
except Exception as e:
    print(f"✗ Error: {e}")

## 8. Integration with Other Aegis Modules

Combine license plate redaction with face and PII redaction.

In [ ]:
# Example: Full anonymization pipeline
# Uncomment and adapt based on your other Aegis modules

# from Aegis.face_redactor import FaceRedactor
# from Aegis.pii_redactor import PIIRedactor

def full_anonymization_pipeline(input_path, output_path):
    """
    Complete anonymization pipeline combining all Aegis modules.
    
    1. License plate redaction
    2. Face redaction
    3. PII redaction
    """
    image = cv2.imread(input_path)
    if image is None:
        raise ValueError(f"Could not read image from {input_path}")
    
    print("Starting full anonymization pipeline...")
    
    # Step 1: Redact license plates
    plate_redactor = LicensePlateRedactor(method='blur')
    image, num_plates = plate_redactor.redact(image, strength=51)
    print(f"✓ Redacted {num_plates} license plate(s)")
    
    # Step 2: Redact faces (integrate your face redactor)
    # face_redactor = FaceRedactor()
    # image, num_faces = face_redactor.redact(image)
    # print(f"✓ Redacted {num_faces} face(s)")
    
    # Step 3: Redact PII text (integrate your PII redactor)
    # pii_redactor = PIIRedactor()
    # image, num_pii = pii_redactor.redact(image)
    # print(f"✓ Redacted {num_pii} PII instance(s)")
    
    # Save final result
    cv2.imwrite(output_path, image)
    print(f"\n✓ Full anonymization complete!")
    print(f"✓ Saved to: {output_path}")
    
    return image

# Example usage:
# full_anonymization_pipeline(
#     '../assets/targets/input.jpg',
#     '../assets/targets/fully_anonymized.jpg'
# )

## Performance Testing

In [ ]:
import time

if image is not None:
    # Benchmark processing time
    start_time = time.time()
    processed, num_plates = redactor.redact(image, strength=51)
    end_time = time.time()
    
    processing_time = (end_time - start_time) * 1000  # Convert to ms
    
    print(f"\nPerformance Metrics:")
    print(f"{'='*40}")
    print(f"Processing time: {processing_time:.2f} ms")
    print(f"Image size: {image.shape[1]}x{image.shape[0]}")
    print(f"Plates detected: {num_plates}")

## Summary

This notebook demonstrated:
- Three different redaction methods (blur, pixelate, black)
- Detection visualization
- Adjustable redaction strength
- Single image and batch processing
- Integration template with other Aegis modules

### Next Steps:
1. Test with your own images
2. Integrate with face and PII redaction
3. Adjust parameters for your use case
4. Consider using the CLI interface via `app.py`